In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sentence_transformers import SentenceTransformer, util

drive.mount('/content/drive', force_remount=True)
df = pd.read_csv('/content/drive/My Drive/Диплом/data.csv')

Mounted at /content/drive


In [ ]:
# Шаг 1: Анализ Частоты Тегов (Tag Frequency Analysis)
def calculate_tag_frequency(tags_list):
    tag_counts = {}  # Создаем словарь для подсчета частоты тегов
    for tags in tags_list:
        # Разбиваем строку тегов на отдельные теги и обновляем их частоту
        for tag in tags.split():
            tag_counts[tag] = tag_counts.get(tag, 0) + 1
    return tag_counts

# Вычисляем частоту тегов для всех записей в столбце 'tags'
tag_counts = calculate_tag_frequency(df['tags'])

# Функция для вычисления балла частоты тегов для каждого запроса
def tag_frequency_score(tags, tag_counts):
    return sum(tag_counts[tag] for tag in tags.split())  # Суммируем частоты тегов для каждого запроса

# Применяем функцию для столбца 'tags' и создаем новый столбец 'TFS' (Tag Frequency Score)
df['TFS'] = df['tags'].apply(lambda x: tag_frequency_score(x, tag_counts))

# Шаг 2: Балл Длины Запроса (Query Length Score, QLS)
def query_length_score(query):
    length = len(query.split())  # Разбиваем запрос на слова и считаем их количество
    if 5 <= length <= 10:
        return 1.0  # Оптимальная длина запроса
    elif length < 5:
        return 0.5  # Слишком короткий запрос
    else:
        return 0.7  # Слишком длинный запрос

# Применяем функцию для столбца 'queries' и создаем новый столбец 'QLS' (Query Length Score)
df['QLS'] = df['queries'].apply(query_length_score)

# Шаг 3: Балл Семантического Сходства (Semantic Similarity Score, SSS)
from sentence_transformers import SentenceTransformer, util

# Загружаем предобученную модель для вычисления эмбеддингов
model = SentenceTransformer('all-MiniLM-L6-v2')

# Популярные запросы, с которыми будем сравнивать запросы из данных
popular_queries = [
    "how to bake cookies",
    "cooking times for meat",
    "safe food storage",
    "how to use spices",
    "basic cooking techniques"
]

# Создаем эмбеддинги для популярных запросов
popular_embeddings = model.encode(popular_queries, convert_to_tensor=True)

# Функция для вычисления балла семантического сходства между запросом и популярными запросами
def semantic_similarity_score(query):
    # Генерируем эмбеддинг для текущего запроса
    query_embedding = model.encode(query, convert_to_tensor=True)
    # Вычисляем косинусное сходство между запросом и популярными запросами
    similarity = util.cos_sim(query_embedding, popular_embeddings).max().item()
    return similarity

# Применяем функцию для столбца 'queries' и создаем новый столбец 'SSS' (Semantic Similarity Score)
df['SSS'] = df['queries'].apply(semantic_similarity_score)

# Шаг 4: Сочетание Оценок
w1, w2, w3 = 0.4, 0.3, 0.3  # Устанавливаем веса для каждой из метрик

# Рассчитываем итоговый балл релевантности на основе взвешенной суммы всех оценок
df['Relevance'] = w1 * df['TFS'] + w2 * df['QLS'] + w3 * df['SSS']

# Шаг 5: Нормализация Релевантности
from sklearn.preprocessing import MinMaxScaler

# Нормализуем итоговые баллы релевантности в диапазон от 0 до 1
scaler = MinMaxScaler()
df['Relevance'] = scaler.fit_transform(df[['Relevance']])

# Сохраняем результат в CSV файл
df.to_csv('relevance_dataset.csv', index=False)

# Выводим столбцы с идентификатором запроса, запросом и его баллом релевантности
print(df[['id', 'queries', 'Relevance']])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

          id                                            queries  Relevance
0          1        How can I get chewy chocolate chip cookies?   0.466042
1          2                How should I cook bacon in an oven?   0.143516
2          3  What is the difference between white and brown...   0.112521
3          4  What is the difference between baking soda and...   0.018683
4          5  In a tomato sauce recipe, how can I cut the ac...   0.238074
...      ...                                                ...        ...
15399  73670                          Poached eggs at altitude?   0.120295
15400  73678  How to thicken buttercream without adding more...   0.007135
15401  73680       Looking for old Italian recipe for chamellas   0.378413
15402  73681     Can i make ice cream with artificial sweetener   0.030713
15403  73683          What are these white 'bits' in my nutella   0.064273

[15404 rows x 3 columns]
